In [1]:
#Notebook written by Miguel de Jesús Martínez Felipe
#For Quantum Computing Mentorship Program (QOSF)
#Assessment Task 1 Multiplier

import logging
import math
from qiskit import *
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy import pi

#from qiskit.circuit.library import QFT

# Quantum Fourier Transform

In [2]:
def qft_rotations(circuit, n):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    circuit.barrier()
    for qubit in range(n):
        circuit.cp(pi/2**(n-qubit), qubit, n)
    qft_rotations(circuit, n)

def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qft(circuit, n):
    """QFT on the first n qubits in circuit"""
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit.decompose()

def qft_inser(circuit,n):
    lenghtCircuit = len(circuit.qubits)
    qft_circ = qft(QuantumCircuit(n,name='QFT'), n)
    circuit.append(qft_circ, circuit.qubits[lenghtCircuit-n:lenghtCircuit])
    return circuit.decompose()

def inverse_qft(circuit, n):
    lenghtCircuit = len(circuit.qubits)
    """Does the inverse QFT on the first n qubits in circuit"""
    qft_circ = qft(QuantumCircuit(n,name='inv_QFT'), n)
    invqft_circ = qft_circ.inverse()
    circuit.append(invqft_circ, circuit.qubits[lenghtCircuit-n:lenghtCircuit])
    return circuit.decompose()

# Draper Adder

In [3]:
def add_qubit(qc, q, qubits_Fourier):
    lenghtCircuit = len(qc.qubits)
    differenceQubits = lenghtCircuit-qubits_Fourier
    for indx, qb in enumerate(range(differenceQubits,lenghtCircuit)):
        qc.cp(pi / (2**((differenceQubits-1) - indx)), q, qb) # Aqui se varía el número elevado dependiendo el resultado
    return qc

def rotationsFourier(qc,qubits_number_A,qubits_Fourier):
    for q in range(0,qubits_number_A):
        power_of_two = 2**q
        for index in range(0,power_of_two):
            a_rotations = add_qubit(QuantumCircuit(len(qc.qubits),name='A'), q,qubits_Fourier)
            qc.append(a_rotations, qc.qubits[0:len(qc.qubits)]) 
            qc.barrier()
        



# Quantum Circuit

In [4]:
def integerToBinary(number):
    binary = bin(number)
    bin_string = str(binary)
    clean_binary = bin_string[2:len(bin_string)]
    return clean_binary

def qubits_A_B(number_1,number_2):
    qubits_A = math.ceil(math.log(number_1,2))
    qubits_B = math.ceil(math.log(number_2,2))
    if number_1 == 1:
        qubits_A = 1
    if number_2 == 1:
        qubits_B = 1
    if number_1 == 2:
        qubits_A = 2
    if number_2 == 2:
        qubits_B = 2
    return qubits_A,qubits_B

def multiplierQuantumCiruit(number_1,number_2):
    qubits_A,qubits_B = qubits_A_B(number_1,number_2)
    qubits_Fourier = qubits_A + qubits_B
    total_qubits = qubits_A + qubits_B + qubits_Fourier
    qc = QuantumCircuit(total_qubits,qubits_Fourier)

    qft_inser(qc,qubits_Fourier)
    for ida, a_value in enumerate(integerToBinary(number_1)[::-1]): # Cambiar el binario del número uno
        if(a_value=='1'):
            qc.x(ida)
                
    for idb, b_value in enumerate(integerToBinary(number_2)[::-1]): # Cambiar el binario del número dos
        if(b_value=='1'):
            qc.x(idb+qubits_A)
    # Rotations with the number_1 
    rotationsFourier(qc,qubits_A,qubits_Fourier)
    
    # Rotations with the number_2
    for idb, b_value in enumerate(integerToBinary(number_2-1)[::-1]):     
        if b_value=='1':
            rotations_b = 2**idb
            for index in range(0,rotations_b):
                rotationsFourier(qc,qubits_A,qubits_Fourier)

    inverse_qft(qc,qubits_Fourier)
    
    qc.measure(range(total_qubits-qubits_Fourier,total_qubits),range(0,qubits_Fourier))
    #qc.draw()
    qc.draw(output='mpl')
    print(qc)
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots = 10)
    counts = job.result().get_counts()
    binary_result = list(counts.keys())[0] 
    decimal = int(binary_result, 2)
    return decimal

# Validations

In [5]:
def validationNumbers(number_1,number_2):
    if (isinstance(number_1, int)) and (isinstance(number_2, int)) and (number_1 is not None) and (number_2 is not None) and (number_1>0) and (number_2>0):
        return True    
    else:
        return False
        
def validationLog(number_1,number_2):
    backend = Aer.get_backend('qasm_simulator')
    backend_qubits = backend.configuration().n_qubits
    logNum_1 = math.ceil(math.log(number_1,2))
    logNum_2 = math.ceil(math.log(number_2,2))
    
    if (logNum_1 + logNum_2) <= (backend_qubits/2):
        return True
    else:
        return False
        
def multiplier(number_1, number_2):
    if validationNumbers(number_1,number_2):
        if validationLog(number_1,number_2):
            return multiplierQuantumCiruit(number_1,number_2)
        else:
            logging.error("limitations of simulation log_2(number_1) + log_2(number_2) should be less than N° of qubits from backend 'qasm_simulator'(30 qubits) ")
    else:
        logging.error("please insert integers greater than 0 and not null values")



In [6]:
"""The limitations of this simulation are only positives integers numbers,
   also log2(number1) + log2(number2) <= 15 since the qasm_simulator has only 30 qubits"""
A = multiplier(123,5)

       ┌───┐  ┌─────┐ ░ ┌─────┐ ░ ┌─────┐ ░ ┌─────┐ ░ ┌─────┐ ░ ┌─────┐ ░ »
 q_0: ─┤ X ├──┤0    ├─░─┤0    ├─░─┤0    ├─░─┤0    ├─░─┤0    ├─░─┤0    ├─░─»
       ├───┤  │     │ ░ │     │ ░ │     │ ░ │     │ ░ │     │ ░ │     │ ░ »
 q_1: ─┤ X ├──┤1    ├─░─┤1    ├─░─┤1    ├─░─┤1    ├─░─┤1    ├─░─┤1    ├─░─»
       └───┘  │     │ ░ │     │ ░ │     │ ░ │     │ ░ │     │ ░ │     │ ░ »
 q_2: ────────┤2    ├─░─┤2    ├─░─┤2    ├─░─┤2    ├─░─┤2    ├─░─┤2    ├─░─»
       ┌───┐  │     │ ░ │     │ ░ │     │ ░ │     │ ░ │     │ ░ │     │ ░ »
 q_3: ─┤ X ├──┤3    ├─░─┤3    ├─░─┤3    ├─░─┤3    ├─░─┤3    ├─░─┤3    ├─░─»
       ├───┤  │     │ ░ │     │ ░ │     │ ░ │     │ ░ │     │ ░ │     │ ░ »
 q_4: ─┤ X ├──┤4    ├─░─┤4    ├─░─┤4    ├─░─┤4    ├─░─┤4    ├─░─┤4    ├─░─»
       ├───┤  │     │ ░ │     │ ░ │     │ ░ │     │ ░ │     │ ░ │     │ ░ »
 q_5: ─┤ X ├──┤5    ├─░─┤5    ├─░─┤5    ├─░─┤5    ├─░─┤5    ├─░─┤5    ├─░─»
       ├───┤  │     │ ░ │     │ ░ │     │ ░ │     │ ░ │     │ ░ │     │ ░ »
 q_6: ─┤ X ├

In [7]:
print(A)

615
